In [1]:
import requests
import json
from bs4 import BeautifulSoup
from openpyxl import Workbook, load_workbook
import re

In [2]:
from tsdavo import translation

In [3]:
def fix_quotes(text):
    return text.replace('“','"').replace('”','"')

In [4]:
def parse_description(text):
    text = fix_quotes(text)
    text = translation(text)
    print('parsing:', text)
    s1 = text.partition('"')
    desc = '"' + s1[-1]
    s2 = s1[0].replace('F.','').replace('op.', '').replace('Op.', '')
    s2 = s2.partition(',')
    return [s2[0].strip(), s2[-1].strip(), desc]

In [ ]:
pre_soviet_cases_url = 'http://archives.kh.gov.ua/?page_id=16179'

In [ ]:
soup = BeautifulSoup(requests.get(pre_soviet_cases_url).text, 'lxml')

In [ ]:
result = []
for tag in soup.find_all('div', attrs = {'id': 'post-16179'}):
    print(tag.name, tag.attrs)
    for tag1 in tag.find_all('li', attrs = {'style': 'text-align: justify;'}):
        a = tag1.find('a')
        #print(a.text, a['href'])
        result.append(parse_description(a.text) + [a['href']])
        #if len(result)>5:
        #    break

In [ ]:
len(result)


In [ ]:
wb = Workbook()

In [ ]:
sheet = wb.active

In [ ]:
for i, item in enumerate(result):
    link = item[-1]
    sheet.append(item)
    row = sheet[i+1]
    for j, v in enumerate(item):
        row[j].value = v
        row[j].hyperlink = link
        row[1].style = 'Hyperlink'

In [ ]:
wb.save('Pre Soviet.xlsx')

In [ ]:
soviet_cases_url = 'http://archives.kh.gov.ua/?page_id=16486'

In [ ]:
soup = BeautifulSoup(requests.get(soviet_cases_url).text, 'lxml')

In [ ]:
def extract_table(table):
    sheet = []
    for tr in table.find_all('tr'):
        row = []
        for td in tr.find_all('td'):
            a = td.find('a')
            if a is not None:
                a = a['href']
            row.append({'text': td.text, 'link': a})
        sheet.append(row)
    return sheet

In [ ]:
def save_table_to_sheet(table, sheet):
    for row in table:
        text_row = [item['text'] for item in row]
        sheet.append(text_row)
        cur = sheet.max_row
        link_row = [item['link'] for item in row]
        the_link = None
        for i, link in enumerate(link_row):
            if link is not None:
                the_link = link
                sheet[cur][i].hyperlink = link
        if the_link is not None:
            for i in range(len(link_row)):
                sheet[cur][i].hyperlink = the_link        

In [ ]:
def translate_column(tbl, col=0):
    for row in tbl:
        row[col]['text'] = translation(row[col]['text'])
    return tbl

In [ ]:
result = []
wb = Workbook()
for tag in soup.find_all('div', attrs = {'id': 'post-16486'}):
    next_sheet = 1
    for table in tag.find_all('table'):
        print(table.name, tag1.attrs)
        tbl = extract_table(table)
        tbl = translate_column(tbl, col=2)
        sheet = wb.create_sheet(f'Table {next_sheet}')
        save_table_to_sheet(tbl, sheet)
        next_sheet = next_sheet + 1
wb.save('Soviet.xlsx')

In [ ]:
for tag in soup.find_all('div', attrs = {'id': 'post-16486'}):
    result = []
    for item in tag.find_all('p'):
        #print(item.name)
        a = item.find('a')
        if a is not None:
            text = translation(fix_quotes(item.text))
            print(text)
            link = a['href']
            result.append({"text": text, "link": link})
            #if len(result) > 5:
            #    break
#wb.save('Soviet.xlsx')

In [ ]:
print(len(result))

In [ ]:
def parse_item(item):
    text = item['text']
    part1 = text.partition('"')
    part2 = part1[0].partition(',')
    fond = part2[0].replace('FR-', '').replace('F. R-', '').strip()
    opus = part2[2].replace('op.', '').strip()
    desc = '"' + part1[2]
    return {'fond': fond, 'opus': opus, 'description': desc, 'link': item['link'], 'text': text}

In [ ]:
parsed_result = [parse_item(item) for item in result]

In [ ]:
def save_items_to_sheet(items, sheet):
    for item in items:
        sheet.append([item['fond'], item['opus'], item['description'], item['text']])
        cur = sheet.max_row
        for i in range(3):
            sheet[cur][i].hyperlink = item['link']

In [ ]:
sheet = wb.create_sheet('Items')
save_items_to_sheet(parsed_result, sheet)        

In [ ]:
wb.save('Soviet.xlsx')

# Augment With Dates, etc.

In [3]:
year_range = re.compile('\d\d\d\d\s*[-–]\s*\d\d\d\d')
single_year = re.compile('\d\d\d\d')
def extract_year_range(text):
    hit = re.search(year_range, text)
    if hit:
        return text[hit.start():hit.end()]
    else:
        hit = re.findall(single_year, text)
        if hit:
            for d in hit:
                d = int(d)
                if d >= 1500 and d < 2030:
                    return f'{d}'
    return ''

In [4]:
keywords = ['pogrom', 'jewish', 'anti-semitism', 'hebrew', 'camps', 'judaic', 'judaism']
pattern = '|'.join(keywords)
exp = re.compile(pattern, re.IGNORECASE)
def keyword_check(text):
    return re.search(exp, text) is not None

In [5]:
def extract_suffix(s):
    result = re.findall(r'\.[^.]+$', s)
    return result[0][1:] if result else ''

In [10]:
wb = load_workbook('Soviet.xlsx')
sheet = wb.active

In [81]:
for i in range(4, 658):
    rng = extract_year_range(sheet[i][2].value)
    jewish = keyword_check(sheet[i][2].value)
    print(rng, jewish)
    sheet.cell(i, 5).value = rng
    sheet.cell(i, 6).value = jewish
    url = sheet[i][2].hyperlink.target
    if url is not None:
        sheet.cell(i, 7).value = extract_suffix(sheet[i][3].hyperlink.target)

1919-1923 False
1919-1923 False
1920-1921 False
1920-1923 False
1920-1922 False
1921 False
1920-1921 False
1921-1923 False
1920-1923 False
1943-1953 False
1920-1922 False
1922-1923 False
1917-1921 False
1920-1922 False
1927-1930 False
1927-1930 False
1922-1924 False
1926 False
1921-1923 False
1920-1921 False
1920-1923 False
1920-1922 False
1919-1921 False
1922-1926 False
1921-1922 False
1921 False
1918-1921 False
1920-1928 False
1943-1950 False
1920-1925 False
1920-1930 False
1924 False
1928-1930 False
1940-1942 False
1919 False
1920 False
1919 False
1919 False
1919 False
1919-1921 False
1919-1922 False
1920-1921 False
1922-1923 False
1923-1925 False
1922-1923 False
1920-1926 False
1919-1920 False
1919-1920 False
1917-1925 False
1919-1925 False
1919 False
1920-1925 False
1918-1920 False
1921 False
1921 False
1922-1923 False
1919-1922 False
1920 False
1920-1921 False
1919 False
1919-1925 False
1919-1922 False
1923-1932 False
1920-1930 False
1922-1930 False
1923-1930 False
1930-1933 Fals

In [82]:
wb.save('Soviet 3.xlsx')

In [6]:
wb = load_workbook('Pre Soviet.xlsx')
sheet = wb.active

In [18]:
def process_sheet(sheet):
    for i in range(4, sheet.max_row+1):
        rng = extract_year_range(sheet[i][2].value)
        jewish = keyword_check(sheet[i][2].value)
        #print(rng, jewish)
        sheet.cell(i, 5).value = rng
        sheet.cell(i, 6).value = jewish
        url = sheet[i][2].hyperlink.target
        if url is not None:
            sheet.cell(i, 7).value = extract_suffix(sheet[i][2].hyperlink.target)

In [19]:
#wb.save('Pre Soviet 4.xlsx')

In [20]:
wb = load_workbook('Soviet.xlsx')
#sheet = wb.active

In [21]:
for sheet in wb:
    print(sheet.title)

Table 1
Table 2
Table 3
Items


In [22]:
for sheet in wb:
    print('processing', sheet.title)
    process_sheet(sheet)

processing Table 1
processing Table 2
processing Table 3
processing Items


In [23]:
wb.save('Soviet 4.xlsx')